In [6]:
import numpy as np
import tensorflow as tf
import os

os.environ['CUDA_VISIBLE_DEVICES']='0'

sess = tf.InteractiveSession()

In [7]:
# 3 x 15 batch

_wv = [[18598, 3436, -1, 16948, 9394, -1, 731, -1, -1, -2, -2, -2, -2, -2, -2],
        [18598, 18541, 7654, -1, 1286, 10087, -1, -1, 731, -1, -1, -2, -2, -2, -2], 
        [ 1321, 18598, 18598, -1, 6604, 5402, 14218, -1, 731, -1, -1, -1, 1, -2, -2]]

_left = [[-1, -1, 0, -1, -1,  3, -1,  5,  2, -2, -2, -2, -2, -2, -2],
         [-1, -1, -1,  1, -1, -1,  4,  3, -1,  7,  0, -2, -2, -2, -2],
         [-1, -1, -1,  1, -1, -1, -1,  5, -1,  7,  4,  3,  0, -2, -2]]

_right = [[-1, -1,  1, -1, -1,  4, -1,  6,  7, -2, -2, -2, -2, -2, -2],
          [-1, -1, -1,  2, -1, -1,  5,  6, -1,  8,  9, -2, -2, -2, -2],
          [-1, -1, -1,  2, -1, -1, -1,  6, -1,  8,  9, 10, 11, -2, -2]]

_label = [[2, 2, 2, 2, 1, 1, 2, 1, 1, -2, -2, -2, -2, -2, -2],
          [2, 2, 2, 2, 1, 2, 1, 1, 2, 1, 1, -2, -2, -2, -2],
          [2, 2, 2, 2, 2, 1, 2, 1, 2, 1, 1, 1, 1, -2, -2]]

_is_leaf = [[1, 1, 0, 1, 1, 0, 1, 0, 0, -2, -2, -2, -2, -2, -2],
            [1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, -2, -2, -2, -2], 
            [1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, -2, -2]]

wv, left, right, label, is_leaf = np.asarray(_wv, dtype=np.int32), np.asarray(_left, dtype=np.int32), np.asarray(_right, dtype=np.int32), \
        np.asarray(_label, dtype=np.int32), np.asarray(_is_leaf, dtype=np.int32)
    
print(wv)

[[18598  3436    -1 16948  9394    -1   731    -1    -1    -2    -2    -2
     -2    -2    -2]
 [18598 18541  7654    -1  1286 10087    -1    -1   731    -1    -1    -2
     -2    -2    -2]
 [ 1321 18598 18598    -1  6604  5402 14218    -1   731    -1    -1    -1
      1    -2    -2]]


In [14]:
bs, len_ts = tf.shape(wv)[0], tf.shape(wv)[1]
sess.run([bs, len_ts])

[3, 15]

In [29]:
# generate rand hidden states

ta = tf.TensorArray(tf.float32, size=0, dynamic_size=True, \
            clear_after_read=False, infer_shape=True)

def forward_loop_body(ta, i):
    rand_states = tf.constant(np.random.random([3, 7]).astype(np.float32)) + tf.to_float(i)
    ta = ta.write(i, rand_states)
    i = i + 1
    return ta, i

forward_loop_cond = lambda ta, i: tf.less(i, len_ts)

ta, _ = tf.while_loop(forward_loop_cond, forward_loop_body, [ta, 0], parallel_iterations=1)
states = ta.stack()

print(sess.run(tf.shape(states)))
print(sess.run(states))

[15  3  7]
[[[  7.49627454e-03   5.82716204e-02   4.48255807e-01   1.36312738e-01
     4.22976047e-01   6.69085026e-01   6.71015024e-01]
  [  2.40481257e-01   3.91616255e-01   5.82712412e-01   5.26218176e-01
     7.93265998e-02   3.87438416e-01   5.82653046e-01]
  [  9.43253994e-01   4.33318228e-01   5.04115701e-01   8.45114827e-01
     4.65849787e-01   5.01698628e-02   3.61610591e-01]]

 [[  1.00749624e+00   1.05827165e+00   1.44825578e+00   1.13631272e+00
     1.42297602e+00   1.66908503e+00   1.67101502e+00]
  [  1.24048126e+00   1.39161623e+00   1.58271241e+00   1.52621818e+00
     1.07932663e+00   1.38743842e+00   1.58265305e+00]
  [  1.94325399e+00   1.43331826e+00   1.50411570e+00   1.84511483e+00
     1.46584976e+00   1.05016983e+00   1.36161065e+00]]

 [[  2.00749636e+00   2.05827165e+00   2.44825578e+00   2.13631272e+00
     2.42297602e+00   2.66908503e+00   2.67101502e+00]
  [  2.24048138e+00   2.39161634e+00   2.58271241e+00   2.52621818e+00
     2.07932663e+00   2.38743830

In [35]:
# get the k-th col

k = 3

print('wvs')
k_wv = tf.slice(wv, [0, k], [-1, 1])
print(sess.run(k_wv))

print
print('ieft ids')
k_left = tf.slice(left, [0, k], [-1, 1])
k_left = tf.reshape(k_left, [-1])
print(sess.run(k_left))

# get states
b = tf.constant([0, 1, 9], dtype=tf.int32)
_states = ta.gather(b)
_ind = tf.tile(tf.expand_dims(tf.range(tf.shape(b)[0]), 1), [1,2])
print(sess.run(_ind))
temp = tf.gather_nd(_states, _ind)
print(sess.run(temp))

wvs
[[16948]
 [   -1]
 [   -1]]

ieft ids
[-1  1  1]
[[[  7.49627454e-03   5.82716204e-02   4.48255807e-01   1.36312738e-01
     4.22976047e-01   6.69085026e-01   6.71015024e-01]
  [  2.40481257e-01   3.91616255e-01   5.82712412e-01   5.26218176e-01
     7.93265998e-02   3.87438416e-01   5.82653046e-01]
  [  9.43253994e-01   4.33318228e-01   5.04115701e-01   8.45114827e-01
     4.65849787e-01   5.01698628e-02   3.61610591e-01]]

 [[  1.00749624e+00   1.05827165e+00   1.44825578e+00   1.13631272e+00
     1.42297602e+00   1.66908503e+00   1.67101502e+00]
  [  1.24048126e+00   1.39161623e+00   1.58271241e+00   1.52621818e+00
     1.07932663e+00   1.38743842e+00   1.58265305e+00]
  [  1.94325399e+00   1.43331826e+00   1.50411570e+00   1.84511483e+00
     1.46584976e+00   1.05016983e+00   1.36161065e+00]]

 [[  9.00749588e+00   9.05827141e+00   9.44825554e+00   9.13631248e+00
     9.42297649e+00   9.66908455e+00   9.67101479e+00]
  [  9.24048138e+00   9.39161587e+00   9.58271217e+00   9.526